In [1]:
import bs4,requests,time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from p_tqdm import p_map
from tqdm import tqdm

base = 'https://www.nomisweb.co.uk'
url = base + '/census/2011/data_finder?msclkid=5c256a10cefb11ecb9a8c95ad9816a91'

options = webdriver.ChromeOptions()
options.add_argument('--headless')


# executable_path param is not needed if you updated PATH
browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)

browser.get(url)
time.sleep(5)
html = browser.page_source


tables = bs4.BeautifulSoup(html, "html.parser")



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/danielellis/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache
/var/folders/h8/t803dxss41g9kj07lkgx95fm0000gn/T/ipykernel_97624/799877897.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)


In [4]:

# for el in tables.find_all('div', class_='datasetItemName'):

items = []
def download_panel(el):
    href  = base+el.find('a')['href']
    
    soup = bs4.BeautifulSoup(requests.get(href).text, "html.parser")
    download = base+soup.find('div', {'id':"download_panel"}).form['action']

    return dict(name = el.text, href=href , download=download , code=download.split('/')[-1].split('.')[0]) 
    
    # break

# items = p_map(download_panel, list(tables.find_all('div', class_='datasetItemName')))
for el in tqdm(tables.find_all('div', class_='datasetItemName')):
    items.append(download_panel(el))

100%|██████████| 755/755 [12:57<00:00,  1.03s/it]


In [5]:
import json 

with open('../static/data/nomis_tables.json', 'w') as f:
    json.dump(items, f, indent=4)

items


[{'name': 'Adults not in employment and dependent children and persons with long-term health problems or disability for all households [KS106EW]',
  'href': 'https://www.nomisweb.co.uk/census/2011/ks106ew',
  'download': 'https://www.nomisweb.co.uk/api/v01/dataset/nm_606_1.bulk.csv',
  'code': 'nm_606_1'},
 {'name': 'Age structure [KS102EW]',
  'href': 'https://www.nomisweb.co.uk/census/2011/ks102ew',
  'download': 'https://www.nomisweb.co.uk/api/v01/dataset/nm_145_1.bulk.csv',
  'code': 'nm_145_1'},
 {'name': 'Age Structure [KS102UK]',
  'href': 'https://www.nomisweb.co.uk/census/2011/ks102uk',
  'download': 'https://www.nomisweb.co.uk/api/v01/dataset/nm_159_1.bulk.csv',
  'code': 'nm_159_1'},
 {'name': 'Car or van availability [KS404EW]',
  'href': 'https://www.nomisweb.co.uk/census/2011/ks404ew',
  'download': 'https://www.nomisweb.co.uk/api/v01/dataset/nm_621_1.bulk.csv',
  'code': 'nm_621_1'},
 {'name': 'Car or van availability [KS404UK]',
  'href': 'https://www.nomisweb.co.uk/cen

## bulk vs data analysis --- ignore from here 


In [ ]:
import pandas as pd
a = pd.read_csv('https://www.nomisweb.co.uk/api/v01/dataset/NM_608_1.data.csv?date=latest&geography=MAKE|Custom|`E07000200;E07000201;E07000202,K04000001&rural_urban=0&measures=20100&select=geography_name,cell_name,obs_value,obs_date')
b = pd.read_csv('https://www.nomisweb.co.uk/api/v01/dataset/NM_608_1.bulk.csv?date=latest&geography=MAKE|Custom|`E07000200;E07000201;E07000202,K04000001&rural_urban=0&measures=20100&select=geography_name,cell_name,obs_value,obs_date')

In [ ]:
a

In [ ]:
b
